In [34]:
import gzip
import sys
from pympler import asizeof
import json

text = "Your text data here"
text_en = text.encode('utf-8')
print( f"size of text: [{text}] : is {sys.getsizeof(text)} bytes" )
print( f"size of encoded text : [{text_en}] : is {sys.getsizeof( text_en )} bytes" )

text_com = gzip.compress(text.encode('utf-8'))
text_en_com = gzip.compress(text_en)
print( sys.getsizeof( text_com))
print(text_com)

print(f"de-compress: {gzip.decompress(text_com).decode()}")

size of text: [Your text data here] : is 68 bytes
size of encoded text : [b'Your text data here'] : is 52 bytes
72
b'\x1f\x8b\x08\x00\xea\xb0qd\x02\xff\x8b\xcc/-R(I\xad(QHI,IT\xc8H-J\x05\x00\xc1\xca\x8c_\x13\x00\x00\x00'
de-compress: Your text data here


In [54]:
file_path = "data.json"
def write_dict_file(data):
    # Open the file in wrte mode
    with open(file_path , "w") as file:
        # Write the dictionary as a JSON string to the file
        json.dump(data, file)

def load_data_file():
    '''load current data and append new data'''
    data = None
    with open(file_path, "r") as file:
        data = json.load(file)
        return data

In [93]:
from serpapi import GoogleSearch

class Authors(object):
  def __init__(self) -> None:
    '''constructor for the class.'''
    
  def search_scholar(self, ):
    '''search author from google scholar...'''
    search = GoogleSearch({
        "engine": "google_scholar",
        "q": "nlp",
        "author": "Mercy Mpinganjira",
        "location": "south africa",
        "api_key": "623e7e74cde13a07079ea444adbbb7b0fd7070796b34b061e21fc1be638c7c9b"
      })
    result = search.get_dict()
    return result['organic_results']

  def search_author(self, author_id: str ):
    '''search result by author'''
    search = GoogleSearch({
      "engine": "google_scholar_author",
      "author_id": author_id,
      "api_key": "623e7e74cde13a07079ea444adbbb7b0fd7070796b34b061e21fc1be638c7c9b"
    })
    result = search.get_dict()
    return result

  def get_author_profile(self, names: str):
    params = {
      "engine": "google_scholar_profiles",
      "mauthors": names,
      "api_key": "623e7e74cde13a07079ea444adbbb7b0fd7070796b34b061e21fc1be638c7c9b"
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    profiles = results["profiles"]
    return profiles

In [94]:
# 
a_name = "Mercy Mpinganjira"
get_author_profile(a_name)



# r = search_author()
# print(f"{r.keys()}")
# r['co_authors']

[{'name': 'Mercy Mpinganjira', 'link': 'https://scholar.google.com/citations?hl=en&user=ry6x3GUAAAAJ', 'serpapi_link': 'https://serpapi.com/search.json?author_id=ry6x3GUAAAAJ&engine=google_scholar_author&hl=en', 'author_id': 'ry6x3GUAAAAJ', 'affiliations': 'Professor, University of Johannesburg', 'email': 'Verified email at uj.ac.za', 'cited_by': 2188, 'interests': [{'title': 'Digital Marketing and Consumer Behaviour', 'serpapi_link': 'https://serpapi.com/search.json?engine=google_scholar_profiles&hl=en&mauthors=label%3Adigital_marketing_and_consumer_behaviour', 'link': 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=label:digital_marketing_and_consumer_behaviour'}], 'thumbnail': 'https://scholar.googleusercontent.com/citations?view_op=small_photo&user=ry6x3GUAAAAJ&citpid=1'}]


In [ ]:
# write to file
# print(result['organic_results'])
available_data = load_data_file()
# print(available_data)
if available_data:
    write_dict_file( result['organic_results'].extend(available_data) )
else:
    write_dict_file( result['organic_results'] )
# write_dict_file( result['organic_results'] )

In [57]:
print(result.keys())
len( result['organic_results'] )

dict_keys(['search_metadata', 'search_parameters', 'search_information', 'organic_results', 'related_searches', 'pagination', 'serpapi_pagination'])


20

In [95]:
d = {'title': 'Understanding mobile marketing adoption intention by South African SMEs: A multi-perspective framework', 'link': 'https://scholar.google.com/citations?view_op=view_citation&hl=en&user=ry6x3GUAAAAJ&citation_for_view=ry6x3GUAAAAJ:TFP_iSt0sucC', 'citation_id': 'ry6x3GUAAAAJ:TFP_iSt0sucC', 'authors': 'DK Maduku, M Mpinganjira, H Duh', 'publication': 'International Journal of Information Management 36 (5), 711-723, 2016', 'cited_by': {'value': 362, 'link': 'https://scholar.google.com/scholar?oi=bibs&hl=en&cites=12942585782408667539', 'serpapi_link': 'https://serpapi.com/search.json?cites=12942585782408667539&engine=google_scholar&hl=en', 'cites_id': '12942585782408667539'}, 'year': '2016'}

d

{'title': 'Understanding mobile marketing adoption intention by South African SMEs: A multi-perspective framework',
 'link': 'https://scholar.google.com/citations?view_op=view_citation&hl=en&user=ry6x3GUAAAAJ&citation_for_view=ry6x3GUAAAAJ:TFP_iSt0sucC',
 'citation_id': 'ry6x3GUAAAAJ:TFP_iSt0sucC',
 'authors': 'DK Maduku, M Mpinganjira, H Duh',
 'publication': 'International Journal of Information Management 36 (5), 711-723, 2016',
 'cited_by': {'value': 362,
  'link': 'https://scholar.google.com/scholar?oi=bibs&hl=en&cites=12942585782408667539',
  'serpapi_link': 'https://serpapi.com/search.json?cites=12942585782408667539&engine=google_scholar&hl=en',
  'cites_id': '12942585782408667539'},
 'year': '2016'}